In [1]:
import itertools
from gurobipy import Model, GRB

def solve_linear_program(with_constraints=True):
    model = Model("Linear_Programming_Example")
    
    attributes = ["A", "B", "C", "D", "E", "F", "G", "H"]
    relations = [["A","B","C","D"], ["A","B","E","F"], ["C","D","E","F"], ["A","B","G","H"], ["C","D","G","H"]]
    fds = [[["E","F","G","H"], ["A","B","C","D"]], [["A","B","C","E"], ["D","F","G","H"]], [["B","C","D","G"], ["A","E","F","H"]]]
    
    attributes = ["x", "y", "z", "u"]
    relations = [["x", "y"], ["y", "z"], ["z", "u"]]
    fds = [[["x", "z"], ["u"]], [["y", "u"], ["x"]], [["x", "z", "y"], ["u", "y"]], [["y", "u", "z"], ["x", "z"]]]
    
    variables = {}
    prob_coeffs = {}
    for i in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, i):
            name = "".join(subset)
            #name = "h(" + name + ")"
            # ub=len(subset)
            x = model.addVar(lb=0, ub= len(subset), name=name)
            variables[frozenset(subset)] = x
            p = model.addVar(lb=0, ub=1, name="p_" + name)
            prob_coeffs[frozenset(subset)] = p
    
    model.setObjective(variables[frozenset(attributes)], GRB.MAXIMIZE)

    for rel in relations:
        model.addConstr(variables[frozenset(rel)] <= 1, name = "Rel_" + "".join(rel))

    for Y, x in fds:
        Yx = Y + x
        model.addConstr(variables[frozenset(Yx)] == variables[frozenset(Y)], "fd_" + "".join(Y) + "->" + "".join(x))
        
    if with_constraints:
        # Consraint two party systems
        for A, B in itertools.combinations(attributes, 2):
            # monotonicity
            model.addConstr(variables[frozenset([A,B])] >= variables[frozenset([A])], name = "Two_" + A)
            model.addConstr(variables[frozenset([A,B])] >= variables[frozenset([B])], name = "Two_" + B)
            # submodularity
            model.addConstr(variables[frozenset([A,B])] <= variables[frozenset([A])] + variables[frozenset([B])], name = "Two_" + A + B)
            
        # Constraint three party systems
        for A, B, C in itertools.combinations(attributes, 3):
            # monotonicity
            model.addConstr(variables[frozenset([A,B,C])] >= variables[frozenset([A,B])], name = "Three_" + A + B)
            model.addConstr(variables[frozenset([A,B,C])] >= variables[frozenset([A,C])], name = "Three_" + A + C)
            model.addConstr(variables[frozenset([A,B,C])] >= variables[frozenset([B,C])], name = "Three_" + B + C)
            # Submodularity
            abc = [A,B,C]
            for j, l in itertools.combinations(abc, 2):
                S = [x for x in abc if x != j and x != l]
                for i in range(1, len(S) + 1):
                    for subset in itertools.combinations(S, i):
                        subset = list(subset)
                        jS = subset + [j]
                        lS = subset + [l]
                        jlS = subset + [j, l]
                        model.addConstr(variables[frozenset(jS)] + variables[frozenset(lS)] - variables[frozenset(subset)] - variables[frozenset(jlS)] >= 0, name="Submodularity")
            
            # Submodularity for those cases that subset S is empty (this reduces to subadditivity)
            for j, l in itertools.combinations(abc, 2):
                model.addConstr(variables[frozenset([j])] + variables[frozenset([l])] - variables[frozenset([j, l])] >= 0, name="Subadditivity")
        
        
    
    #for attr in attributes:
    #    model.addConstr(variables[frozenset([attr])] <= 1, name = "Attr_" + attr)
    
    if False:
        for i in range(4, len(attributes) + 1):
            for subset in itertools.combinations(attributes, i):
                term = 0
                for elem in subset:
                    term += prob_coeffs[frozenset([elem])]*variables[frozenset(elem)]
                model.addConstr(variables[frozenset(subset)] >= term, name = "Super_" + "".join(subset))
                # Loop over two elements at once
                term = 0
                if i % 2 == 0:
                    for k in range(0, i, 2):
                        elem1 = subset[k]
                        elem2 = subset[k+1]
                        term += prob_coeffs[frozenset([elem1, elem2])]*variables[frozenset([elem1, elem2])]
                    model.addConstr(variables[frozenset(subset)] <= term, name = "Super_" + "".join(subset))
                else:
                    for k in range(0, i - 3, 2):
                        elem1 = subset[k]
                        elem2 = subset[k+1]
                        term += prob_coeffs[frozenset([elem1, elem2])]*variables[frozenset([elem1, elem2])]
                    term += prob_coeffs[frozenset([subset[-3], subset[-2], subset[-1]])]*variables[frozenset([subset[-3], subset[-2], subset[-1]])]
                    model.addConstr(variables[frozenset(subset)] <= term, name = "Super_" + "".join(subset))
        
            
    # Save model to lp file
    model.write("Linear_Programming_Example2.lp")

    # Optimize the model
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")
    
    return model.objVal

# Call the function
value1 = solve_linear_program()
value2 = solve_linear_program(with_constraints=False)

print("====================================")
print(value1, value2)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 61 rows, 30 columns and 161 nonzeros
Model fingerprint: 0x60fbfa12
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 26 rows and 19 columns
Presolve time: 0.00s
Presolved: 35 rows, 11 columns, 92 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0030000e+00   1.201800e+01   0.000000e+00      0s
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H w